In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from fbprophet import Prophet
from datetime import datetime
from sklearn import preprocessing
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.metrics import r2_score
from sklearn import preprocessing
from mlxtend.regressor import StackingRegressor
%matplotlib inline 

In [2]:
successfulVends_raw = pd.read_csv("successfulVENDS2019.csv")
deposit_raw = pd.read_csv("deposit2019.csv")
loan_raw = pd.read_csv("loan2019.csv")

In [3]:
deposit_raw= deposit_raw.rename(index=str,columns={"transamount":"amount","systemtranstime":"time"})

In [12]:
deposit= deposit_raw[['billrefnumber','amount','time']]

In [13]:
deposit['time']=pd.to_datetime(deposit.time)

In [14]:
d = deposit.groupby(['billrefnumber'])

In [26]:
d.groups

{'0074836': Index(['48388', '48877'], dtype='object'),
 '01450001308': Index(['83720', '85486'], dtype='object'),
 '01450001316': Index(['33304'], dtype='object'),
 '01450002009': Index(['20726'], dtype='object'),
 '01450003551': Index(['20364'], dtype='object'),
 '01450005036': Index(['12294'], dtype='object'),
 '01450005366': Index(['22488'], dtype='object'),
 '01450005390': Index(['8710'], dtype='object'),
 '01450005408': Index(['84026'], dtype='object'),
 '01450006299': Index(['22490'], dtype='object'),
 '01450006315': Index(['23343', '35210'], dtype='object'),
 '01450009111': Index(['19118', '33666'], dtype='object'),
 '01450011745': Index(['33368'], dtype='object'),
 '01450012131': Index(['5885', '43369', '43481', '78950'], dtype='object'),
 '01450012560': Index(['53334', '60489'], dtype='object'),
 '01450014574': Index(['55477'], dtype='object'),
 '01450014764': Index(['21663'], dtype='object'),
 '01450015381': Index(['40436'], dtype='object'),
 '01450017064': Index(['51874', '8

In [6]:
deposit = deposit.groupby(['billrefnumber','time']).sum()


In [11]:
deposit.drop(index='time',level=1)

amount
billrefnumber time                       
0074836       2019-02-14 10:41:52   300.0
              2019-02-14 18:47:58   300.0
01450001308   2019-03-26 03:18:21    50.0
              2019-03-26 12:37:23   300.0
01450001316   2019-01-30 18:48:35   300.0
01450002009   2019-01-24 05:03:55   200.0
01450003551   2019-01-24 02:49:33   500.0
01450005036   2019-01-17 16:09:33  1000.0
01450005366   2019-01-24 10:09:23   300.0
01450005390   2019-01-13 06:43:06  1000.0
01450005408   2019-03-26 05:46:09   500.0
01450006299   2019-01-24 10:09:28   100.0
01450006315   2019-01-24 12:46:22    50.0
              2019-02-01 08:19:24   100.0
01450009111   2019-01-23 15:02:52   250.0
              2019-01-31 01:59:14   150.0
01450011745   2019-01-30 18:57:48   100.0
01450012131   2019-01-09 13:54:32   500.0
              2019-02-08 16:34:28   300.0
              2019-02-08 17:53:10   200.0
              2019-03-20 20:23:03  1000.0
01450012560   2019-02-19 15:15:47   300.0
              2019-02-26 11:38:29   320.0
01450014574   2019-02-22 07:44:21   500.0
01450014764   2019-01-24 08:15:27    20.0
01450015381   2019-02-05 16:16:18  1000.0
01450017064   2019-02-17 21:31:17   200.0
              2019-03-24 16:13:11   200.0
01450018773   2019-03-16 09:35:06   500.0
01450019201   2019-03-26 06:40:20   100.0
...                                   ...
e37156484141  2019-01-24 13:19:10   180.0
e37166710030  2019-01-24 10:03:34    50.0
              2019-01-24 14:03:17    60.0
              2019-01-24 14:05:45    20.0
              2019-01-30 16:33:42    90.0
              2019-01-30 16:37:08    20.0
              2019-01-30 16:47:33    20.0
              2019-01-30 16:52:04    20.0
              2019-01-30 16:55:48    40.0
              2019-01-30 17:00:05    20.0
              2019-01-30 17:04:00    20.0
              2019-01-30 17:20:59    20.0
              2019-01-30 17:22:57    20.0
              2019-01-30 17:32:41    20.0
              2019-01-30 18:09:26    20.0
              2019-01-30 18:19:06    40.0
              2019-01-31 12:17:36    20.0
              2019-01-31 12:26:27    20.0
e37167805912  2019-03-01 18:37:45    20.0
e37167925546  2019-03-19 17:43:05   180.0
e37169271501  2019-01-24 04:05:20   100.0
e37169328707  2019-01-27 17:53:04   200.0
e37170619276  2019-01-23 21:48:33   100.0
e37174075202  2019-01-20 09:06:17   100.0
              2019-01-20 09:09:40    20.0
e37174882698  2019-01-24 12:37:58   100.0
e37174972655  2019-02-26 16:22:13    50.0
e37183023565  2019-01-24 11:16:34   500.0
e54600296732  2019-02-07 05:45:31   700.0
e54600471681  2019-01-23 05:43:04    20.0

[85699 rows x 1 columns]